In [42]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Concatenate, Embedding, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import Sequence
from tensorflow.keras.regularizers import l2, l1, l1_l2
from tensorflow.keras.initializers import RandomUniform, he_normal,he_uniform
import math

In [43]:
import pandas as pd
col_list = ["U_ID", "P_ID", "RATING"]
dataset = pd.read_csv('rating.csv',usecols=col_list, sep='\t',dtype='int')

In [44]:
#to get the unique user ids from dataset
user_ids = dataset['U_ID'].values
unique_ids = np.unique(user_ids)
id_dict = {id: counter for counter, id in enumerate(unique_ids)}
#enumerate(unique_ids)
dataset['U_ID'] = dataset['U_ID'].apply(lambda x: id_dict[x])

In [45]:
#to get the unique product ids from dataset
product_ids = dataset['P_ID'].values
unique_ids = np.unique(product_ids)
#create a dictionary of users with key value from 0 to #users
id_dict = {id: counter for counter, id in enumerate(unique_ids)}
dataset['P_ID'] = dataset['P_ID'].apply(lambda x: id_dict[x])

In [46]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(dataset, test_size=0.2, random_state=42)
n_users = len(dataset.U_ID.unique())
n_movies = len(dataset.P_ID.unique())
n_latent_factors=20

In [47]:

#movie input and embedding
movie_input = keras.layers.Input(shape=[1],name='Item')
movie_embedding = keras.layers.Embedding(n_movies, n_latent_factors, 
                                          embeddings_initializer="uniform", 
                                          embeddings_regularizer=l2(1e-6), 
                                          embeddings_constraint='NonNeg',
                                          name='Movie-Embedding')(movie_input)
movie_vec = keras.layers.Flatten(name='FlattenMovies')(movie_embedding)
 
#user input and embedding
user_input = keras.layers.Input(shape=[1],name='User')
user_embedding = keras.layers.Embedding(n_users, n_latent_factors, 
                                          embeddings_initializer="uniform", 
                                          embeddings_regularizer=l2(1e-6), 
                                          embeddings_constraint='NonNeg',
                                          name='User-Embedding')(user_input)
user_vec = keras.layers.Flatten(name='FlattenUsers')(user_embedding)


In [48]:
concat = keras.layers.concatenate([movie_vec, user_vec])
 
mlp = concat
for i in range(6,-1,-1):
    if i == 0:
      mlp = Dense(8**i, activation='sigmoid', kernel_initializer='glorot_normal',
                  name="output")(mlp)
    else:
      mlp = Dense(8*2**i, activation='relu', kernel_initializer='he_uniform')(mlp)
      if i > 5:
        mlp = BatchNormalization()(mlp)
        mlp = Dropout(0.05)(mlp)
 
model = Model(inputs=[user_input, movie_input], outputs=[mlp])
 
model.compile('adamax', loss='mean_squared_error', metrics=['mae', 'mse'])

In [49]:
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
train, test = train_test_split(dataset, test_size=0.2, random_state=42)
#early_stop = EarlyStopping(monitor='val_output_loss', min_delta = 0.0001, patience=10)
 
#traindatagenerator = DataGenerator(train, train.RATING, shuffle=False)
 
#history = model.fit(traindatagenerator,epochs=10, verbose=1)
history = model.fit([train.U_ID, train.P_ID], train.RATING, epochs=10, verbose=1)

Epoch 1/10


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


 143/7103 [..............................] - ETA: 4:04 - loss: 11.4255 - mae: 3.2143 - mse: 11.4252

KeyboardInterrupt: ignored